In [15]:
from sqlmodel import select
from database import get_session
from models import Role, User

In [18]:
def insert_role(name: str, description: str):
    session = next(get_session())  # Get the actual session from the generator
    try:
        role = Role(name=name, description=description)
        session.add(role)
        session.commit()
        session.refresh(role)
        return role
    finally:
        session.close()

    
def insert_user(name:str, username: str, email: str, password: str, role_id: int):
        session = next(get_session())
        user = User(name=name, username=username, email=email, password_hash=password, role_id=role_id)
        session.add(user)
        session.commit()
        session.refresh(user)
        print(f"User Added: {user}")
        return user

In [19]:
# Insert roles
admin_role = insert_role("Admin", "Administrator with full access.")
editor_role = insert_role("Editor", "Can edit and manage content.")

# # Insert users linked to roles
# insert_user("Alice Johnson", "alicej", "alice@example.com", "hashed_password1", admin_role.id)
# insert_user("Bob Smith", "bobsmith", "bob@example.com", "hashed_password2", editor_role.id)

In [21]:
def get_user(username: str):
        session = next(get_session())  # Get the actual session from the generator
        statement = select(User).where(User.username == username)
        user = session.exec(statement).first()
        return user
    
user = get_user("alicej")

In [22]:
user

User(name='Alice Johnson', username='alicej', password_hash='$2b$12$k8WfgAwGj5pvKcaKpI1H0Om523lmft1wF0374Ncygz6vj2WAsIFEq', id=1, email='alice@example.com', role_id=1)

In [23]:
user.name

'Alice Johnson'

In [ ]:
## To have access to role of user: use Eager Loading
from sqlalchemy.orm import selectinload
from sqlmodel import select

def get_user(username: str):
        session = next(get_session())  # Get the actual session from the generator
        statement = (
            select(User)
            .where(User.username == username)
            .options(selectinload(User.role))  # Eager load the role
        )
        user = session.exec(statement).first()
        return user

user = get_user("alicej")

if user and user.role:
    print(f"User: {user.name}, Role: {user.role.name}, Description: {user.role.description}")
else:
    print("User or role not found.")

User: Alice Johnson, Role: Admin, Description: Administrator with full access.


## Advanced Python Tutorial: Context Manager and, Decorators
The `contextlib` module provides utilities for working with context managers, which manage resources like files or database connections using the `with` statement.

In [26]:
from contextlib import contextmanager

@contextmanager
def open_file(file, mode):
    f = open(file, mode)
    try:
        yield f
    finally:
        f.close()

with open_file('example.txt', 'w') as f:
    f.write('Hello, World!')

Here, `contextlib` helps manage file opening and closing, avoiding manual cleanup.

In [28]:
from sqlmodel import Session
from database import engine

In [29]:
@contextmanager
def get_session():
    session = Session(engine)
    try:
        yield session
        session.commit()   # Commit if all operations succeed
    except Exception as e:
        session.rollback() # Rollback if any operation fails
        raise e
    finally:
        session.close()    # Always close the session

In [ ]:
with get_session() as session:
    new_user = User(name="Taofeek", email="taofeek@example.com")
    session.add(new_user)

In [ ]:
@contextmanager
def get_session():
    with Session(engine) as session:
        yield session


`@contextmanager`: is a decorator that simplifies the process of creating a context manager. When you decorate a function with `@contextmanager`, it becomes a context manager function.

- Turns the `get_session` function into a context manager, allowing it to be used with a `with` statement.

`with Session(engine) as session`:

- This line creates a new database session using `Session(engine)` (commonly from SQLAlchemy).
- The `with` statement ensures that session.close() is automatically called, releasing database resources.

`yield session`:

- Temporarily **returns the session** to the calling block so you can run queries.
- After the `with` block completes (successfully or with an error), the session is **closed automatically**